# Read the README [in the repository](https://github.com/jzlotek/cs583-final) before continuing


In [3]:
#
# download_dataset.py
#

import requests
import urllib
import os
import sys
from zipfile import ZipFile

OUTDIR='./dataset'

def download_file_from_google_drive(id, destination):
    URL = f"https://storage.googleapis.com/isl-datasets/SID/{id}"

    session = requests.Session()

    response = session.get(URL, stream = True, timeout=None)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True, timeout=None)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 10000
    downloaded = 0
    total = 26926678016.00 * 1.1 / 100.00
    sys.stdout.write("\r0.00% downloaded")
    sys.stdout.flush()

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                downloaded+=len(chunk)
                if downloaded % (CHUNK_SIZE*50) == 0:
                    sys.stdout.write('\r%0.2f%% downloaded' % (downloaded/(total)))
    sys.stdout.write("\r100.00% downloaded")

# Ensure directory exists
if not os.path.exists(OUTDIR):
  os.mkdir(OUTDIR)

filepath = OUTDIR+'/Sony.zip'
# Too much data, download the smaller dataset for now
print('Downloading Sony data... (25GB)')
download_file_from_google_drive('Sony.zip', filepath)

fileout = OUTDIR+'/Sony'
# Always unzip to reset directory
print('\nUnzipping, this will take a while...')
print('Usage of storage should be around 117GB')
ZipFile(filepath).extractall(path=OUTDIR)

    
print("Data ready in %s" % (fileout))

100.00% downloaded
Unzipping, this will take a while...
Usage of storage should be around 117GB
Data ready in ./dataset/Sony


Data is downloaded, time to train the model.

In [0]:
!pip install keras
!pip install rawpy
!pip install imageio

In [4]:
#
# better training
#

import glob
import numpy as np
import os
import rawpy

from keras import backend as k
from keras.layers import \
Input, Conv2D, LeakyReLU, MaxPooling2D, \
UpSampling2D, Conv2DTranspose, Concatenate, \
ZeroPadding2D, UpSampling3D
from keras.callbacks import LearningRateScheduler
import keras.layers.advanced_activations
import keras.optimizers
from keras.utils import plot_model

import tensorflow as tf

# Constants
target_dir = './dataset/Sony/long/'
input_dir = './dataset/Sony/short/'

ps = 512 # patch size

# Helper Functions
def pack_raw(raw):
    # pack Bayer image to 4 channels
    im = raw.raw_image_visible.astype(np.float32)
    im = np.maximum(im - 512, 0) / (16383 - 512)  # subtract the black level

    im = np.expand_dims(im, axis=2)
    img_shape = im.shape
    H = img_shape[0]
    W = img_shape[1]

    out = np.concatenate((im[0:H:2, 0:W:2, :],
                          im[0:H:2, 1:W:2, :],
                          im[1:H:2, 1:W:2, :],
                          im[1:H:2, 0:W:2, :]), axis=2)
    return out
  
def load_batches(batch_size):   
    train_fns = glob.glob(target_dir + '0*.ARW') 
    train_ids = np.array([int(os.path.basename(train_fn)[0:5]) \
                          for train_fn in train_fns])
    
    train_ids = np.random.permutation(train_ids)
    
    total_length = len(train_ids)
    
    while True:
        batch_start = 0
        batch_end = batch_size

        while batch_start < total_length:
            limit = min(batch_end, total_length)  
            batch_ids = train_ids[batch_start:limit]
            
            # Intialize return values
            inputs, targets = np.empty((batch_size,ps,ps,4)), np.empty((batch_size,ps*2,ps*2,3))
            index = 0
            for image_id in batch_ids:
              
                # Get an random input filename
                input_files = glob.glob(input_dir + '%05d_00*.ARW' % image_id)
                input_path = None
                if len(input_files) <= 1:
                    input_path = input_files[0]
                else :
                    input_path = input_files[np.random.randint(0, len(input_files) - 1)]
                input_fn = os.path.basename(input_path)

                # Get the matching target name
                target_files = glob.glob(target_dir + '%05d_00*.ARW' % image_id)
                target_path = target_files[0]
                target_fn = os.path.basename(target_path)
                          
                # Calculate amplification ratio
                input_exposure = float(input_fn[9:-5])
                target_exposure = float(target_fn[9:-5])
                ratio = min(target_exposure / input_exposure, 300)
                
                # Load image into memory
                raw = rawpy.imread(input_path)
                input_image = \
                  np.expand_dims(pack_raw(raw), axis=0) * ratio

                target_image = rawpy.imread(target_path)
                target_image = target_image.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)
                target_image = np.expand_dims(np.float32(target_image / 65535.0), axis=0)

                # Take a random patch from each image
                H = input_image.shape[1]
                W = input_image.shape[2]

                xx = np.random.randint(0, W - ps)
                yy = np.random.randint(0, H - ps)
                input_patch = input_image[:, yy:yy + ps, xx:xx + ps, :]
                target_patch = target_image[:, yy * 2:yy * 2 + ps * 2, xx * 2:xx * 2 + ps * 2, :]
        
                # Compute random alterations to the images
                if np.random.randint(2, size=1)[0] == 1:  # random flip
                    input_patch = np.flip(input_patch, axis=1)
                    target_patch = np.flip(target_patch, axis=1)
                if np.random.randint(2, size=1)[0] == 1:
                    input_patch = np.flip(input_patch, axis=2)
                    target_patch = np.flip(target_patch, axis=2)
                if np.random.randint(2, size=1)[0] == 1:  # random transpose
                    input_patch = np.transpose(input_patch, (0, 2, 1, 3))
                    target_patch = np.transpose(target_patch, (0, 2, 1, 3))

                input_patch = np.minimum(input_patch, 1.0)

                # Add to return arrays
                inputs[index] = input_patch
                targets[index] = target_patch
                
                del input_patch
                del target_patch
                
                index += 1

            yield (inputs,targets,(batch_start/batch_size))

            del inputs
            del targets
            
            batch_start += batch_size   
            batch_end += batch_size
            
        return

def main():
    # Backend Config
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = 0.5
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))
    k.set_image_data_format('channels_last')
    
    # Define the model
    inputs = Input(shape=(None,None,4,), dtype="float32",
                name="Inputs")

    x = Conv2D(filters=32, kernel_size=(3,3),
            name="conv_1_1", padding='same')(inputs)
    x = LeakyReLU(alpha=0.2, name="conv_1_act_1")(x)
    x = Conv2D(filters=32, kernel_size=(3,3),
            name="conv_1_2", padding='same')(x)
    x1 = LeakyReLU(alpha=0.2, name="conv_1_act_2")(x)
    x = MaxPooling2D(pool_size=(2, 2), padding='same',
            name="conv_1_pool")(x1)

    x = Conv2D(filters=64, kernel_size=(3,3),
            name="conv_2_1", padding='same')(x)
    x = LeakyReLU(alpha=0.2, name="conv_2_act_1")(x)
    x = Conv2D(filters=64, kernel_size=(3,3),
            name="conv_2_2", padding='same')(x)
    x2 = LeakyReLU(alpha=0.2, name="conv_2_act_2")(x)
    x = MaxPooling2D(pool_size=(2, 2), padding='same',
            name="conv_2_pool")(x2)

    x = Conv2D(filters=128, kernel_size=(3,3),
            name="conv_3_1", padding='same')(x)
    x = LeakyReLU(alpha=0.2, name="conv_3_act_1")(x)
    x = Conv2D(filters=128, kernel_size=(3,3),
            name="conv_3_2", padding='same')(x)
    x3 = LeakyReLU(alpha=0.2, name="conv_3_act_2")(x)
    x = MaxPooling2D(pool_size=(2, 2), padding='same',
            name="conv_3_pool")(x3)

    x = Conv2D(filters=256, kernel_size=(3,3),
            name="conv_4_1", padding='same')(x)
    x = LeakyReLU(alpha=0.2, name="conv_4_act_1")(x)
    x = Conv2D(filters=256, kernel_size=(3,3),
            name="conv_4_2", padding='same')(x)
    x4 = LeakyReLU(alpha=0.2, name="conv_4_act_2")(x)
    x = MaxPooling2D(pool_size=(2, 2), padding='same',
            name="conv_4_pool")(x4)

    x = Conv2D(filters=512, kernel_size=(3,3),
            name="conv_5_1", padding='same')(x)
    x = LeakyReLU(alpha=0.2, name="conv_5_act_1")(x)
    x = Conv2D(filters=512, kernel_size=(3,3),
            name="conv_5_2", padding='same')(x)
    x5 = LeakyReLU(alpha=0.2, name="conv_5_act_2")(x)
    
    x5 = Conv2DTranspose(filters=256, kernel_size=(2,2),
            strides=(2,2), padding='same')(x5)
    x = Concatenate(axis=3)([x5,x4])
    x = Conv2D(filters=256, kernel_size=(3,3),
            name="conv_7_1", padding='same')(x)
    x = LeakyReLU(alpha=0.2, name="conv_7_act_1")(x)
    x = Conv2D(filters=256, kernel_size=(3,3),
            name="conv_7_2", padding='same')(x)
    x6 = LeakyReLU(alpha=0.2, name="conv_7_act_2")(x)

    x6 = Conv2DTranspose(filters=128, kernel_size=(2,2),
            strides=(2,2), padding='same')(x6)
    x = Concatenate(axis=3)([x6,x3])
    x = Conv2D(filters=128, kernel_size=(3,3),
            name="conv_8_1", padding='same')(x)
    x = LeakyReLU(alpha=0.2, name="conv_8_act_1")(x)
    x = Conv2D(filters=128, kernel_size=(3,3),
            name="conv_8_2", padding='same')(x)
    x7 = LeakyReLU(alpha=0.2, name="conv_8_act_2")(x)

    x7 = Conv2DTranspose(filters=64, kernel_size=(2,2),
            strides=(2,2), padding='same')(x7)
    x = Concatenate(axis=3)([x7,x2])
    x = Conv2D(filters=64, kernel_size=(3,3),
            name="conv_9_1", padding='same')(x)
    x = LeakyReLU(alpha=0.2, name="conv_9_act_1")(x)
    x = Conv2D(filters=64, kernel_size=(3,3),
            name="conv_9_2", padding='same')(x)
    x8 = LeakyReLU(alpha=0.2, name="conv_9_act_2")(x)

    x8 = Conv2DTranspose(filters=32, kernel_size=(2,2),
            strides=(2,2), padding='same')(x8)
    x = Concatenate(axis=3)([x8,x1])
    x = Conv2D(filters=32, kernel_size=(3,3),
            name="conv_10_1", padding='same')(x)
    x = LeakyReLU(alpha=0.2, name="conv_10_act_1")(x)
    x = Conv2D(filters=32, kernel_size=(3,3),
            name="conv_10_2", padding='same')(x)
    x = LeakyReLU(alpha=0.2, name="conv_10_act_2")(x)

    x = Conv2D(filters=3, kernel_size=(1,1), activation=None,
            name="conv_to_image")(x)

    outputs = UpSampling2D(size=(2,2))(x)

    # Create the model
    model = keras.Model(inputs=inputs, outputs=outputs, name='ltsitd')

    # Get info about the model
    if False:
        model.summary(line_length=150)
        
        plot_model(model, to_file='model.png')
        from google.colab import files
        files.download('model.png')
    
    batch_size = 2          # Number of images to load into memory at once (RAM/GPU Memory restricted)
    total_epochs = 11       # Number of training rounds per batch

    # Drop the learning rate by a factor of 10 halfway through
    def step_decay(epoch):
        lrate = 1e-4
        
        if epoch > total_epochs // 2:
          lrate = 1e-5
        
        return lrate
      
    lrate = LearningRateScheduler(step_decay)
    callback_list = [lrate]
    
    # Build the model
    model.compile(loss='mae',          # mean absolute error for loss
                  optimizer='Adam',    # Adam optimizer
                  metrics=['acc'])     # Print accuracy info during training

    # Train the model
    for inputs, targets, batch in load_batches(batch_size):
        print('\n------------------------------------------------------------------------------------')
        print("Training batch #%d" % batch)
        model.fit(inputs, 
                  targets,
                  steps_per_epoch=batch_size,
                  epochs=total_epochs,
                  callbacks=callback_list, # Call the step_decay function
                  verbose=1)
    
    # Save model
    model.save('model.h5') 
    print('Saved model -> model.h5')
    del model 

# Run the program
main()



------------------------------------------------------------------------------------
Training batch #0
Epoch 1/11
2/2 [==============================] - 4s 2s/step - loss: 0.2254 - acc: 0.1785
Epoch 2/11
2/2 [==============================] - 1s 289ms/step - loss: 0.2208 - acc: 0.1732
Epoch 3/11
2/2 [==============================] - 1s 297ms/step - loss: 0.2158 - acc: 0.1659
Epoch 4/11
2/2 [==============================] - 1s 290ms/step - loss: 0.2106 - acc: 0.1572
Epoch 5/11
2/2 [==============================] - 1s 283ms/step - loss: 0.2050 - acc: 0.1482
Epoch 6/11
2/2 [==============================] - 1s 292ms/step - loss: 0.1985 - acc: 0.1403
Epoch 7/11
2/2 [==============================] - 1s 294ms/step - loss: 0.1926 - acc: 0.1358
Epoch 8/11
2/2 [==============================] - 1s 292ms/step - loss: 0.1917 - acc: 0.1355
Epoch 9/11
2/2 [==============================] - 1s 289ms/step - loss: 0.1907 - acc: 0.1353
Epoch 10/11
2/2 [==============================] - 1s 288ms/st

2/2 [==============================] - 1s 271ms/step - loss: 0.0728 - acc: 0.8543
Epoch 4/11
2/2 [==============================] - 1s 288ms/step - loss: 0.0721 - acc: 0.6501
Epoch 5/11
2/2 [==============================] - 1s 298ms/step - loss: 0.0731 - acc: 0.2603
Epoch 6/11
2/2 [==============================] - 1s 277ms/step - loss: 0.0704 - acc: 0.1520
Epoch 7/11
2/2 [==============================] - 1s 270ms/step - loss: 0.0674 - acc: 0.1604
Epoch 8/11
2/2 [==============================] - 1s 300ms/step - loss: 0.0670 - acc: 0.1725
Epoch 9/11
2/2 [==============================] - 1s 281ms/step - loss: 0.0666 - acc: 0.1955
Epoch 10/11
2/2 [==============================] - 1s 276ms/step - loss: 0.0662 - acc: 0.2314
Epoch 11/11
2/2 [==============================] - 1s 282ms/step - loss: 0.0657 - acc: 0.2826

------------------------------------------------------------------------------------
Training batch #8
Epoch 1/11
2/2 [==============================] - 1s 300ms/step - lo

2/2 [==============================] - 1s 283ms/step - loss: 0.0554 - acc: 0.8675
Epoch 7/11
2/2 [==============================] - 1s 272ms/step - loss: 0.0560 - acc: 0.8666
Epoch 8/11
2/2 [==============================] - 1s 292ms/step - loss: 0.0559 - acc: 0.8666
Epoch 9/11
2/2 [==============================] - 1s 304ms/step - loss: 0.0557 - acc: 0.8667
Epoch 10/11
2/2 [==============================] - 1s 274ms/step - loss: 0.0555 - acc: 0.8668
Epoch 11/11
2/2 [==============================] - 1s 280ms/step - loss: 0.0552 - acc: 0.8670

------------------------------------------------------------------------------------
Training batch #15
Epoch 1/11
2/2 [==============================] - 1s 280ms/step - loss: 0.1123 - acc: 0.8561
Epoch 2/11
2/2 [==============================] - 1s 298ms/step - loss: 0.0979 - acc: 0.8698
Epoch 3/11
2/2 [==============================] - 1s 273ms/step - loss: 0.0719 - acc: 0.8738
Epoch 4/11
2/2 [==============================] - 1s 270ms/step - l

2/2 [==============================] - 1s 288ms/step - loss: 0.0625 - acc: 0.8432
Epoch 10/11
2/2 [==============================] - 1s 276ms/step - loss: 0.0618 - acc: 0.8433
Epoch 11/11
2/2 [==============================] - 1s 276ms/step - loss: 0.0610 - acc: 0.8435

------------------------------------------------------------------------------------
Training batch #22
Epoch 1/11
2/2 [==============================] - 1s 296ms/step - loss: 0.0819 - acc: 0.7308
Epoch 2/11
2/2 [==============================] - 1s 275ms/step - loss: 0.0678 - acc: 0.7310
Epoch 3/11
2/2 [==============================] - 1s 272ms/step - loss: 0.0578 - acc: 0.7310
Epoch 4/11
2/2 [==============================] - 1s 292ms/step - loss: 0.0530 - acc: 0.7310
Epoch 5/11
2/2 [==============================] - 1s 288ms/step - loss: 0.0523 - acc: 0.7310
Epoch 6/11
2/2 [==============================] - 1s 280ms/step - loss: 0.0533 - acc: 0.7309
Epoch 7/11
2/2 [==============================] - 1s 278ms/step - l

2/2 [==============================] - 1s 293ms/step - loss: 0.1378 - acc: 0.6806
Epoch 2/11
2/2 [==============================] - 1s 278ms/step - loss: 0.1236 - acc: 0.6937
Epoch 3/11
2/2 [==============================] - 1s 270ms/step - loss: 0.1174 - acc: 0.6939
Epoch 4/11
2/2 [==============================] - 1s 292ms/step - loss: 0.1170 - acc: 0.6940
Epoch 5/11
2/2 [==============================] - 1s 285ms/step - loss: 0.1180 - acc: 0.6940
Epoch 6/11
2/2 [==============================] - 1s 275ms/step - loss: 0.1159 - acc: 0.6941
Epoch 7/11
2/2 [==============================] - 1s 271ms/step - loss: 0.1121 - acc: 0.6952
Epoch 8/11
2/2 [==============================] - 1s 301ms/step - loss: 0.1114 - acc: 0.6956
Epoch 9/11
2/2 [==============================] - 1s 275ms/step - loss: 0.1105 - acc: 0.6959
Epoch 10/11
2/2 [==============================] - 1s 278ms/step - loss: 0.1096 - acc: 0.6962
Epoch 11/11
2/2 [==============================] - 1s 281ms/step - loss: 0.1086 

...

2/2 [==============================] - 1s 279ms/step - loss: 0.0415 - acc: 0.4666
Epoch 11/11
2/2 [==============================] - 1s 291ms/step - loss: 0.0414 - acc: 0.4708

------------------------------------------------------------------------------------
Training batch #51
Epoch 1/11
2/2 [==============================] - 1s 295ms/step - loss: 0.1195 - acc: 0.7799
Epoch 2/11
2/2 [==============================] - 1s 270ms/step - loss: 0.0944 - acc: 0.8255
Epoch 3/11
2/2 [==============================] - 1s 271ms/step - loss: 0.0716 - acc: 0.8249
Epoch 4/11
2/2 [==============================] - 1s 287ms/step - loss: 0.0612 - acc: 0.8175
Epoch 5/11
2/2 [==============================] - 1s 284ms/step - loss: 0.0578 - acc: 0.8242
Epoch 6/11
2/2 [==============================] - 1s 270ms/step - loss: 0.0575 - acc: 0.8377
Epoch 7/11
2/2 [==============================] - 1s 274ms/step - loss: 0.0566 - acc: 0.8465
Epoch 8/11
2/2 [==============================] - 1s 297ms/step - lo

2/2 [==============================] - 1s 287ms/step - loss: 0.1046 - acc: 0.5772
Epoch 2/11
2/2 [==============================] - 1s 300ms/step - loss: 0.0986 - acc: 0.5768
Epoch 3/11
2/2 [==============================] - 1s 280ms/step - loss: 0.0901 - acc: 0.5757
Epoch 4/11
2/2 [==============================] - 1s 280ms/step - loss: 0.0838 - acc: 0.5736
Epoch 5/11
2/2 [==============================] - 1s 281ms/step - loss: 0.0815 - acc: 0.5690
Epoch 6/11
2/2 [==============================] - 1s 300ms/step - loss: 0.0809 - acc: 0.5603
Epoch 7/11
2/2 [==============================] - 1s 278ms/step - loss: 0.0795 - acc: 0.5496
Epoch 8/11
2/2 [==============================] - 1s 274ms/step - loss: 0.0789 - acc: 0.5483
Epoch 9/11
2/2 [==============================] - 1s 288ms/step - loss: 0.0781 - acc: 0.5472
Epoch 10/11
2/2 [==============================] - 1s 289ms/step - loss: 0.0771 - acc: 0.5462
Epoch 11/11
2/2 [==============================] - 1s 277ms/step - loss: 0.0759 

2/2 [==============================] - 1s 275ms/step - loss: 0.0708 - acc: 0.7170
Epoch 5/11
2/2 [==============================] - 1s 272ms/step - loss: 0.0700 - acc: 0.6811
Epoch 6/11
2/2 [==============================] - 1s 302ms/step - loss: 0.0693 - acc: 0.6601
Epoch 7/11
2/2 [==============================] - 1s 281ms/step - loss: 0.0688 - acc: 0.6606
Epoch 8/11
2/2 [==============================] - 1s 279ms/step - loss: 0.0687 - acc: 0.6631
Epoch 9/11
2/2 [==============================] - 1s 278ms/step - loss: 0.0686 - acc: 0.6674
Epoch 10/11
2/2 [==============================] - 1s 303ms/step - loss: 0.0684 - acc: 0.6729
Epoch 11/11
2/2 [==============================] - 1s 271ms/step - loss: 0.0681 - acc: 0.6788

------------------------------------------------------------------------------------
Training batch #66
Epoch 1/11
2/2 [==============================] - 1s 276ms/step - loss: 0.0383 - acc: 0.5251
Epoch 2/11
2/2 [==============================] - 1s 292ms/step - l

2/2 [==============================] - 1s 281ms/step - loss: 0.0863 - acc: 0.8994
Epoch 8/11
2/2 [==============================] - 1s 290ms/step - loss: 0.0857 - acc: 0.8997
Epoch 9/11
2/2 [==============================] - 1s 294ms/step - loss: 0.0849 - acc: 0.9003
Epoch 10/11
2/2 [==============================] - 1s 273ms/step - loss: 0.0841 - acc: 0.9009
Epoch 11/11
2/2 [==============================] - 1s 277ms/step - loss: 0.0833 - acc: 0.9016

------------------------------------------------------------------------------------
Training batch #73
Epoch 1/11
2/2 [==============================] - 1s 305ms/step - loss: 0.0371 - acc: 0.8139
Epoch 2/11
2/2 [==============================] - 1s 283ms/step - loss: 0.0379 - acc: 0.8149
Epoch 3/11
2/2 [==============================] - 1s 271ms/step - loss: 0.0406 - acc: 0.8152
Epoch 4/11
2/2 [==============================] - 1s 293ms/step - loss: 0.0421 - acc: 0.8154
Epoch 5/11
2/2 [==============================] - 1s 300ms/step - l

2/2 [==============================] - 1s 273ms/step - loss: 0.0452 - acc: 0.8610
Epoch 11/11
2/2 [==============================] - 1s 275ms/step - loss: 0.0449 - acc: 0.8609

------------------------------------------------------------------------------------
Training batch #80
Epoch 1/11
2/2 [==============================] - 1s 273ms/step - loss: 0.0318 - acc: 0.8735
Epoch 2/11
2/2 [==============================] - 1s 278ms/step - loss: 0.0294 - acc: 0.5614
Epoch 3/11
2/2 [==============================] - 1s 304ms/step - loss: 0.0273 - acc: 0.4848
Epoch 4/11
2/2 [==============================] - 1s 272ms/step - loss: 0.0256 - acc: 0.4642
Epoch 5/11
2/2 [==============================] - 1s 279ms/step - loss: 0.0241 - acc: 0.4432
Epoch 6/11
2/2 [==============================] - 1s 290ms/step - loss: 0.0232 - acc: 0.4274
Epoch 7/11
2/2 [==============================] - 1s 290ms/step - loss: 0.0229 - acc: 0.4225
Epoch 8/11
2/2 [==============================] - 1s 271ms/step - lo

In [0]:
from google.colab import files
files.download('model.h5')

In [6]:
from keras.models import load_model
import imageio
import rawpy
import numpy as np

DIR = 'dataset/Sony/short/'

INPUTS=[
    '00182_03_0.04s.ARW',
    '20211_01_0.033s.ARW',
    '00186_01_0.033s.ARW',
    '00222_05_0.04s.ARW',
    '00209_00_0.033s.ARW',
    '00232_03_0.04s.ARW', 
    '20201_03_0.04s.ARW',
    '10054_01_0.04s.ARW',
    '00207_01_0.04s.ARW',
    '00215_00_0.033s.ARW'
]

OUT_FMT='test_%s.jpg'

# Loads the saved model
model = load_model('model.h5')

for image in INPUTS:
    iid = image[0:5]
    raw = rawpy.imread(DIR+image)

    i = np.expand_dims(pack_raw(raw), axis=0) * 200
    i = np.minimum(i, 1.0)

    # Runs the model on a new input image
    output = model.predict(i)

    output = np.minimum(np.maximum(output, 0), 1)
    output = output[0, :, :, :]

    output *= 255

    imageio.imwrite(OUT_FMT % (iid), np.array(output, dtype=np.uint8))
    print('saved %s' % (OUT_FMT % (iid)))

#     files.download(DIR+image)
#     files.download(OUT_FMT % (iid))
    

saved test_00182.jpg
saved test_20211.jpg
saved test_00186.jpg
saved test_00222.jpg
saved test_00209.jpg
saved test_00232.jpg
saved test_20201.jpg
saved test_10054.jpg
saved test_00207.jpg
saved test_00215.jpg
